In [53]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import numpy as np

load_dotenv()

path = "./BaseTargetList.xlsx"
df = pd.read_excel(path)

In [54]:
df2 = df.groupby(['Buyer', 'ClientSubSector'])['ClientSubSector'].count().to_frame(name = 'Ndeals')
df2

Ndeals
Buyer                                    ClientSubSector                                           
123 Milhas                               Varejo Especializado                                     1
180° Seguros                             IT Services                                              1
220 Capital                              Distribuição industrial                                  1
                                         Indústria Pesada, Máquinas e Equipamentos                1
                                         Serviços Terceirizados (limpeza, segurança e ca...       1
...                                                                                             ...
zeppelin                                 Locação de Ativos                                        2
ziff davis                               Meios de Pagamento                                       1
Água Da Serra Industrial De Bebidas Ltda Bebidas                                                  1
Évora                                    Embalagens                                               1
Órama                                    Fintech                                                  1

[8785 rows x 1 columns]

In [55]:
# # Número de NDAs assinados por esse buyer para deals neste sub-setor; 
df2['SignedNDADate'] = df.groupby(['Buyer', 'ClientSubSector'])['SignedNDADate'].count()
df2

Ndeals  \
Buyer                                    ClientSubSector                                              
123 Milhas                               Varejo Especializado                                     1   
180° Seguros                             IT Services                                              1   
220 Capital                              Distribuição industrial                                  1   
                                         Indústria Pesada, Máquinas e Equipamentos                1   
                                         Serviços Terceirizados (limpeza, segurança e ca...       1   
...                                                                                             ...   
zeppelin                                 Locação de Ativos                                        2   
ziff davis                               Meios de Pagamento                                       1   
Água Da Serra Industrial De Bebidas Ltda Bebidas                                                  1   
Évora                                    Embalagens                                               1   
Órama                                    Fintech                                                  1   

                                                                                             SignedNDADate  
Buyer                                    ClientSubSector                                                    
123 Milhas                               Varejo Especializado                                            0  
180° Seguros                             IT Services                                                     0  
220 Capital                              Distribuição industrial                                         0  
                                         Indústria Pesada, Máquinas e Equipamentos                       0  
                                         Serviços Terceirizados (limpeza, segurança e ca...              1  
...                                                                                                    ...  
zeppelin                                 Locação de Ativos                                               0  
ziff davis                               Meios de Pagamento                                              0  
Água Da Serra Industrial De Bebidas Ltda Bebidas                                                         0  
Évora                                    Embalagens                                                      0  
Órama                                    Fintech                                                         0  

[8785 rows x 2 columns]

In [56]:
# Número de Gos do Buyer nesse setor
df2["NumberOfGosPerBuyerInSubSector"] = df.groupby(['Buyer', 'ClientSubSector'])['PassedReason'].count()

In [57]:
# Número de No Gos, por razão de tamanho, neste sub-setor; 
TooSmall = df.groupby(['Buyer', 'ClientSubSector'])['PassedReason'].apply(lambda x: (x == 'Too Small').sum())
TooBig = df.groupby(['Buyer', 'ClientSubSector'])['PassedReason'].apply(lambda x: (x == 'Too Big').sum())
df2['NumberOfGosPerSizeInSubSector'] = TooSmall + TooBig

In [58]:
#Número de NBOs enviadas pelo buyer para deals neste sub-setor
df2['NumberOfNBOperDealsInSubSector']  = df.groupby(['Buyer', 'ClientSubSector'])['SubmittedNBODate'].count()

In [59]:
#Média do Tamanho dos Deals
df2['AverageSizePerDealInSubSector'] = df.groupby(['Buyer', 'ClientSubSector'])['EstimatedDealSize'].mean()

In [60]:
#Mediana do Tamanho dos Deals
df2['MedianSizePerDealInSubSector'] = df.groupby(['Buyer', 'ClientSubSector'])['EstimatedDealSize'].median()

In [61]:
#Data da última NBO enviada pelo Buyer
df['SubmittedNBODate'] = pd.to_datetime(df['SubmittedNBODate']).dt.date
df['SubmittedNBODate'] = pd.to_datetime(df['SubmittedNBODate'])
df2['NBOLastDatePerDealInSubSector'] = df.groupby(['Buyer', 'ClientSubSector'])['SubmittedNBODate'].max()

In [62]:
#Números de Deals fechados por esse buyer nesse setor
df2['DealClosePerClientSector'] = df.groupby(['Buyer', 'ClientSubSector'])['Deal Close Date'].count()

In [63]:
#Data do último Deal fechado nesse sub-setor
df['Deal Close Date'] = pd.to_datetime(df['Deal Close Date'])
df2['LastDateCloseDealPerInSubSector'] = df.groupby(['Buyer','ClientSubSector'])['Deal Close Date'].max()

In [64]:
# df[df["Buyer"]=='EB Capital']

In [65]:
#Contato do buyer, considerando a data de contato mais recente; 
df_sorted = df.sort_values(by=['Buyer', 'LastContactDate'], ascending=[True, False])
df2['MostRecentContact'] = df_sorted.groupby(['Buyer', 'ClientSubSector'])['BuyerContact'].first()

In [66]:
df['IsEqual'] = df['BuyerSubSector'] == df['ClientSubSector']
df2['IsEqual'] = df.groupby(['Buyer', 'ClientSubSector'])['IsEqual'].any()

In [67]:
df2.loc[df2.index.get_level_values('Buyer') == 'Advent']


Ndeals  \
Buyer  ClientSubSector                                              
Advent Asset Management                                         8   
       Bancos Tradicionais                                      1   
       Dental & Medical devices (produção e distribuição)       3   
       Distribuição industrial                                  2   
       Embalagens                                               3   
       Ensino Superior                                          1   
       Fertilizantes & Defensivos                               1   
       Fintech                                                  2   
       Frutas, Legumes e Verduras                               1   
       IT Services                                              1   
       Industria e distribuição de Químicos & Ingredie...       3   
       Indústria Automotiva                                     1   
       Indústria Farmacêutica                                   1   
       Indústria Pesada, Máquinas e Equipamentos                2   
       Locação de Ativos                                        1   
       Logistica                                                2   
       Lácteos, sorvetes e refrigerados                         1   
       Marketing Services                                       2   
       Materiais Básicos e Produtos para Construção (I...       1   
       Meios de Pagamento                                       2   
       Nutrição animal                                          1   
       Produtos de Uso Pessoal e Cosméticos                     2   
       Restaurantes & Delivery                                  1   
       Seguradoras & Corretoras de Seguros                      1   
       Serviços Terceirizados (limpeza, segurança e ca...       3   
       Software                                                 1   
       Varejo Especializado                                     3   

                                                           SignedNDADate  \
Buyer  ClientSubSector                                                     
Advent Asset Management                                                3   
       Bancos Tradicionais                                             0   
       Dental & Medical devices (produção e distribuição)              1   
       Distribuição industrial                                         1   
       Embalagens                                                      0   
       Ensino Superior                                                 1   
       Fertilizantes & Defensivos                                      0   
       Fintech                                                         1   
       Frutas, Legumes e Verduras                                      0   
       IT Services                                                     1   
       Industria e distribuição de Químicos & Ingredie...              2   
       Indústria Automotiva                                            1   
       Indústria Farmacêutica                                          0   
       Indústria Pesada, Máquinas e Equipamentos                       0   
       Locação de Ativos                                               0   
       Logistica                                                       2   
       Lácteos, sorvetes e refrigerados                                1   
       Marketing Services                                              1   
       Materiais Básicos e Produtos para Construção (I...              0   
       Meios de Pagamento                                              1   
       Nutrição animal                                                 0   
       Produtos de Uso Pessoal e Cosméticos                            1   
       Restaurantes & Delivery                                         0   
       Seguradoras & Corretoras de Seguros                             0   
       Serviços Terceirizados (limpeza, segurança e ca...              1   
     

In [68]:
# caminho_arquivo_excel = './arquivoTest.csv'
caminho_arquivo_excel = './arquivoTest.csv'
# Exporte o DataFrame para o arquivo Excel
df2.to_csv(caminho_arquivo_excel)
# df2.to_excel(caminho_arquivo_excel)

In [69]:
db_user = 'postgres'
db_password = 'AmbienteDev'
db_host = 'localhost'
db_port = 5432
db_database = 'projetoetl'
connection_url = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}'
engine = create_engine(connection_url)
tabela = 'targetTest'
df2.to_sql(tabela, engine, if_exists='replace')

785

In [71]:
df2.tail(50)

Ndeals  \
Buyer                                    ClientSubSector                                              
dLocal                                   Bancos Tradicionais                                      1   
                                         IT Services                                              3   
                                         Meios de Pagamento                                       1   
eClerx                                   IT Services                                              2   
eGain                                    IT Services                                              1   
ePlus                                    IT Services                                              1   
eToro                                    Software                                                 1   
eventX                                   IT Services                                              1   
giift                                    Meios de Pagamento                                       1   
green dot                                Meios de Pagamento                                       1   
i-Med                                    IT Services                                              1   
i3 Verticals                             Bancos Tradicionais                                      1   
iBeta                                    IT Services                                              1   
iClinic                                  Varejo Especializado                                     1   
iFood                                    IT Services                                              5   
                                         Marketing Services                                       1   
                                         Marketplace Diversos                                     2   
                                         Varejo Especializado                                     4   
iTechArt Group                           IT Services                                              1   
inEvent                                  Consulting                                               1   
                                         IT Services                                              1   
inbetta                                  Embalagens                                               1   
iugu                                     Serviços Terceirizados (limpeza, segurança e ca...       1   
ixigo                                    Varejo Especializado                                     1   
luxottica                                Varejo Especializado                                     1   
m12                                      Agricultura de precisão/implementos                      1   
nCino                                    IT Services                                              3   
nVent Eletric plc                        Indústria Pesada, Máquinas e Equipamentos                1   
nift                                     Meios de Pagamento                                       1   
prizeout                                 Meios de Pagamento                                       1   
qwikcilver                               Meios de Pagamento                                       1   
recharge                                 Meios de Pagamento                                       1   
rymax                                    Meios de Pagamento                                       1   
sprinklr                                 IT Services                                              1   
stagwell group                           IT Services                                              3   
                                         Marketing Services                                       3   
                                         Software                                                 1   
svs                                      Meios de Pagamento                 